# Data Exploration

### Import

In [ ]:
#import
import pandas as pd

#controls number of columns being printed
pd.set_option('max_columns', None)

#read CSV
df = pd.read_csv('', header='None')

### Set Header

In [ ]:
#set headers
headers=["header_1", "header_2", "header_3"]
df.columns = headers

#get headers
print(df.columns)

### See Dataframe

In [ ]:
#check first 5 rows
df.head()

#check last 5 rows
df.tail()

### Export Dataset

In [ ]:
df.to_csv("automobile.csv", index=False)

### Data Types

In [ ]:
#check datatypes
df.dtypes

In [ ]:
#get statistical summary
df.describe()
#more advance summary
df.describe(include="all")

In [ ]:
#more concise summary
df.info

# Data Wrangling

### Handle Missing Values
How to deal with missing data?
##### drop data
- drop the whole row
- drop the whole column

##### replace data
- replace it by mean
- replace it by frequency
- replace it based on other functions

In [ ]:
# missing data check
missing_data = df.isnull()
missing_data.head()

#missing data counter
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

#isna.count
df.isna().sum()
    
# dropna
df.dropna(subset=["price"], axis=0, inplace=True)

#change column types then get mean
avg_norm_loss = df["normalized-losses"].astype(float).mean(axis=0)

#replace NaN
df["normalized-losses"].replace(np.nan, avg_norm_loss, inplace=True)

#to see which values are present in a particular column
df["num-of-doors"].value_counts()
#get the most frequency values showed
df["num-of-doors"].value_counts().idxmax()
df["num-of-doors"].replace(np.nan, "four", inplace=True)

#reset index after dropping rows
df.reset_index(drop=True, inplace=True)

In [ ]:
#changing types
df[["bore", "stroke"]] = df[["bore", "stroke"]].astype("float")
df[["normalized-losses"]] = df[["normalized-losses"]].astype("int")
df[["price"]] = df[["price"]].astype("float")
df[["peak-rpm"]] = df[["peak-rpm"]].astype("float")

#last check
df.dtypes

In [ ]:
#rename cplumn
df["highway-mpg"] = 235/df["highway-mpg"]
df.rename(columns={'"highway-mpg"': 'highway-L/100km'}, inplace=True)
df.head()

### Data Normalization

In [ ]:
#Simple Feature Scaling
df["length"] = df["length"]/df["length"].max()
#Min Max Feature Scaling
df["length"] = (df["length"]-df["length"].min())/(df["length"].max()-df["length"].min())
#Z-Score
df["length"] = (df["length"]-df["length"].mean())/df["length"].std()

### Binning

In [ ]:
#Apply Histogram Code
%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot
plt.pyplot.hist(df["price"])

# set x/y labels and plot title
plt.pyplot.xlabel("price")
plt.pyplot.ylabel("count")
plt.pyplot.title("price bins")

In [ ]:
# grouping values
bins = np.linspace(min(df["price"]), max(df["price"]), 4)
group_names = ["Low", "Medium", "High"]
df["price-binned"] = pd.cut(df["price"], bins, labels=group_names, include_lowest=True)
df[["price", "price-binned"]].head()
#apply histogram but with the binned version

### One Hot Encoding

In [ ]:
#convert to 0,1 values
dummy_variable_1 = pd.get_dummies(df['fuel'])
dummy_variable_1.rename(columns={'fuel-type-diesel':'gas', 'fuel-type-diesel':'diesel'}, inplace=True)
dummy_variable_1.head()

# merge data frame "df" and "dummy_variable_1" 
df = pd.concat([df, dummy_variable_1], axis=1)

# drop original column "fuel-type" from "df"
df.drop("fuel-type", axis = 1, inplace=True)

# Exploratory Data Analysis

In [ ]:
# get fast statistic summary
df.describe()

# get count number
drive_wheels_counts = df["drive-wheels"].value_counts()

In [ ]:
#DESCRIPTIVE ANALYSIS
#1. boxplot to see distribution and outlier
#2. scatter plot to see relationship between 2 variables (predictor, target)

In [ ]:
#GROUPING DATA
df_test = df[['drive-wheels','body-style','price']]
df_grp = df_test.groupby(['drive-wheels', 'body-style'], as_index=False).mean()
df_grp

In [ ]:
#PIVOT
df_pivot = df_grp.pivot(index='drive-wheels', columns='body-style')

<img src="./pandas_pivot1.png">

In [ ]:
# install
%%capture
! pip install seaborn

#import
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
#Heatmap = see relationship in multiple variables
plt.pcolor(df_pivot, cmap='RdBu')
plt.colorbar()
plt.show()

<img src="./heatmap1.png">

In [ ]:
# advanced
fig, ax = plt.subplots()
im = ax.pcolor(grouped_pivot, cmap='RdBu')

#label names
row_labels = grouped_pivot.columns.levels[1]
col_labels = grouped_pivot.index

#move ticks and labels to the center
ax.set_xticks(np.arange(grouped_pivot.shape[1]) + 0.5, minor=False)
ax.set_yticks(np.arange(grouped_pivot.shape[0]) + 0.5, minor=False)

#insert labels
ax.set_xticklabels(row_labels, minor=False)
ax.set_yticklabels(col_labels, minor=False)

#rotate label if too long
plt.xticks(rotation=90)

fig.colorbar(im)
plt.show()

<img src="./heatmap2.png">

In [ ]:
#ANOVA(Analysis of Variance) = finding correlation between different groups of a categorical variable
#ex. Average price of different vehicle makes.
#returns:
#1. F-test score = calculates the reatio of variation within each of the sample group means. Bigger score = highly correlate
#2. p-value > 0.05 means null hyphoteses is not accepted. Score < 0.5 = good
df_anova = df[["make","price"]]
grouped_anova = df_anova.groupby(["make"])

#anova components:
#1. get group => to get values of the method group
grouped_anova.get_group('subaru')['price']
#2. f_oneway => get f-test score and p-value
anova_results_1 = stats.f_oneway(grouped_anova.get_group("honda")["price"], grouped_anova.get_group("subaru")["price"], grouped_anova.get_group("mercedes")["price"])

In [ ]:
#Correlation = measure to what extent different variables are interdependent
#Correlation doens't imply causation
#ex:
#1. Lung cancer -> smoking
#2. rain -> umbrella
#returns: positive, negative, weak, strong, no correlation

#THIS IS FOR NUMERICAL VARIABLES
sns.regplot(x="engine-size", y="price", data=df)
plt.ylim(0,)

# seek correlation value after visualisation:
df[['feature','target']].corr()

In [ ]:
#THIS IS FOR CATEGORICAL VARIABLES (object/int data types allowed)
#use boxplot. Prevent overlapping boxes
sns.boxplot(x="body-style", y="price", data=df)

#describe for categorical
df.describe(include=['object'])

#check how many units of each variable we have. Note: we are not using double bracket, value_counts works for pandas series (not pandas df)
engine_loc_counts = df['engine-location'].value_counts().to_frame()
engine_loc_counts.rename(columns={'engine-location': 'value_counts'}, inplace=True)
engine_loc_counts.index.name = 'engine-location'
engine_loc_counts.head(10)

<img src="./value_count.png">

In [ ]:
#Another correlation implementation: Pearson Correlation
#Aim: measure the strength of the correlation between two features.
#consists of:
#1. correlation coefficient: linearity test. (+1) strong positive relationship; (-1) strong negative relationship; (0) no relationship
#2. p-value: statistical significance test. The smaller the better (threshold < 0.05) else (>0.1) no correlation.

from scipy import stats

pearson_coef, p_value = stats.pearsonr(df['housepower'], df['price'])
print("The Pearson Correlation Coefficient is", pearson_coef, " with a P-value of P =", p_value)

### Basic of Grouping

In [ ]:
#see different categories
df['drive-wheels'].unique()

#assign to variables
df_group_one = df[['drive-wheels','body-style','price']]

#grouping results
df_group_one = df_group_one.groupby(['drive-wheels'], as_index=False).mean()
df_group_one

<img src="./groupby.png">

In [ ]:
# grouping multiple variables is also alllowed
df_gptest = df[['drive-wheels','body-style','price']]
grouped_test1 = df_gptest.groupby(['drive-wheels','body-style'],as_index=False).mean()
grouped_test1

<img src="./groupby2.png">

In [ ]:
# ALTERNATIVES, you'll never be wrong with pivot tables
grouped_pivot = grouped_test1.pivot(index='drive-wheels',columns='body-style')
grouped_pivot

<img src="./pivot.png">

In [ ]:
# to fill the missing values with 0
grouped_pivot = grouped_pivot.fillna(0)
grouped_pivot

# Model Development

### Simple Linear Regression

y= b0 + b1x
- y = target / dependent variable
- x = predictor / independent variable
- b0 = intercept
- b1 = slope

noise = small random value added 

In [ ]:
#choose model
lm = LinearRegression()
lm

#choose dependent & independent variables
X = df[['highway-mpg']]
y = df['price']

#start train
lm.fit(X,y)

#predict
Yhat=lm.predict(X)
Yhat[0:5]

In [ ]:
#check intercept and slope
lm.intercept_
lm.coef_

### Multiple Linear Regression

$$
Yhat = a + b_1 X_1 + b_2 X_2 + b_3 X_3 + b_4 X_4
$$

In [ ]:
X = df[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']]
y = df['price']
lm.fit(X,y)

lm.intercept_
lm.coef_

### Model Evaluation with Visualization

In [ ]:
# import the visualization package: seaborn
import seaborn as sns
%matplotlib inline 

In [ ]:
#visualize regression plot
width = 12
height = 10
plt.figure(figsize=(width, height))
sns.regplot(x="highway-mpg", y="price", data=df)
plt.ylim(0,)

<img src="./regression_plot.png">

##### What we can see here?
how scattered the data points around the regression line? <- variance.

# visualize residual plot => a good way to visualize the variance of the data
- what is residual? the difference between the observed value (y) and the predicted value (Yhat).
- when we look at a regression plot, the residual is the distance from the data point to the fitted regression line.

- residual plot?? graph that shows the residuals on the vertical y-axis and the independent variable on horizontal x-axis.

- what to see?? <b>look at the spread of the residuals.</b> If the points in a residual plot are randomly spread out, then a linear model is appropriate for the data.

- why?? randomly spread out residuals means that the variance is constant, and the linear model is a good fit for this data.

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))
sns.residplot(df['highway-mpg'], df['price'])
plt.show()

<img src="./residual_plot.png">

#### What is the plot tell us?
We can see from this residual plot that the residuals are not randomly spread around the x-axis, which leads us to believe that maybe a non-linear model is more appropriate for this data.

### Visualize Linear Regression
We can't visualize it with regression or residual plot <b> rather use the distribution of the fitted values and the actual values</b>

In [ ]:
# first make a prediction
Y_hat = lm.predict(Z)

#visualize
plt.figure(figsize=(10, 12))

ax1 = sns.distplot(df['price'], hist=False, color="r", label="Actual Value")
sns.distplot(Yhat, hist=False, color="b", label="Fitted Values" , ax=ax1)

plt.title('Actual vs Fitted Values for Price')
plt.xlabel('Price (in dollars)')
plt.ylabel('Proportion of Cars')

plt.show()
plt.close()

<img src="./visual_mlr.png">

We can see that the fitted values are reasonably close to the actual values, since the two distributions overlap a bit. However, there is definitely some room for improvement.

### Polynomial Regression and Pipelines
- Polynomial regression is a <b>particular case</b> of the general <b>linear regression or multiple linear regression models.</b>
- We get non-linear relationships by <i>squaring</i> or <i>setting higher-order terms of the predictor variables</i>.

* Different orders of polynomial regression:

<center><b>Quadratic - 2nd order</b></center>
$$
Yhat = a + b_1 X^2 +b_2 X^2 
$$


<center><b>Cubic - 3rd order</b></center>
$$
Yhat = a + b_1 X^2 +b_2 X^2 +b_3 X^3\\
$$


<center><b>Higher order</b>:</center>
$$
Y = a + b_1 X^2 +b_2 X^2 +b_3 X^3 ....\\
$$

<p>We saw earlier that a linear model did not provide the best fit while using highway-mpg as the predictor variable. Let's see if we can try fitting a polynomial model to the data instead.</p>
<p>We will use the following function to plot the data:</p>

In [ ]:
def PlotPolly(model, independent_variable, dependent_variabble, Name):
    x_new = np.linspace(15, 55, 100)
    y_new = model(x_new)

    plt.plot(independent_variable, dependent_variabble, '.', x_new, y_new, '-')
    plt.title('Polynomial Fit with Matplotlib for Price ~ Length')
    ax = plt.gca()
    ax.set_facecolor((0.898, 0.898, 0.898))
    fig = plt.gcf()
    plt.xlabel(Name)
    plt.ylabel('Price of Cars')

    plt.show()
    plt.close()
    
# lets get the variables
x = df['highway-mpg']
y = df['price']

# Here we use a polynomial of the 3rd order (cubic) 
f = np.polyfit(x, y, 3)
p = np.poly1d(f)
print(p)

<img src="./polynomial_function.png">

In [ ]:
# in case you want to see np.polyfit result
np.polyfit(x, y, 3)

<img src="./polynomial_function2.png">

In [ ]:
#lets plot the function
PlotPolly(p, x, y, 'highway-mpg')

<img src="./polynomial_cubic_res.png">

### The Analytical Expression for Multiple Polynomial
For example for second order (degree=2) polynomial with 2 variables is given by:

$$
Yhat = a + b_1 X_1 +b_2 X_2 +b_3 X_1 X_2+b_4 X_1^2+b_5 X_2^2
$$

In [ ]:
#import
from sklearn.preprocessing import PolynomialFeatures

# we create a polynomial object for degree 2
pr=PolynomialFeatures(degree=2)
pr

#fit
Z_pr=pr.fit_transform(Z)

#original data shape (201,4)
Z.shape

#fitted data shape (201,15)
Z_pr.shape

### Pipeline
- Data pipelines simplify the steps of processing the data. 
- We use the module Pipeline to create a pipeline. 
- We also use StandardScaler as a step in our pipeline.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
Input=[('scale',StandardScaler()), ('polynomial', PolynomialFeatures(include_bias=False)), ('model',LinearRegression())]

In [ ]:
pipe=Pipeline(Input)
pipe

In [ ]:
pipe.fit(Z,y)

In [ ]:
ypipe=pipe.predict(Z)
ypipe[0:4]

### Measure for In-Sample Evaluation
When evaluating our models, not only visualize the result but also want to measure quantitatively how accurate the model is.
<p>Two very important measures that are often used in Statistics to determine the accuracy of a model are:</p>
<ul>
    <li><b>R^2 / R-squared</b> => or Coefficient of determination, is a measure to indicate how close the data is to the fitted regression line. The value is in percentage of variation of the target variable (y) that's explained by a linear model.</li>
    <li><b>Mean Squared Error (MSE)</b> => measures the average of the squares of errors, that is, the difference between actual value (y) and the estimated value (ŷ)</li>
</ul>

In [ ]:
# R^2
lm.fit(df[['horsepower_fit']], df['price'])
# Find the R^2
print('The R-square is: ', lm.score(X, Y)) # 0.760xxxx.

In [ ]:
#R^2 alternatives
from sklearn.metrics import r2_score

r_squared = r2_score(y, p(x))
print('The R-square value is: ', r_squared) # <- the close to -1/1 the better, the closer to zero = bad

We can say 76% of the variation of the price is explained by the simple linear model "horsepower_fit"

In [ ]:
#now lets calculate the MSE
Yhat=lm.predict(X)
print('The output of the first four predicted value is: ', Yhat[0:4])

In [ ]:
#calculate MSE
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(df['price'], Yhat)
print('The mean square error of price and predicted value is: ', mse) #15021126.025174143 <- the smaller the better

### Prediction and Decision Making
#### Prediction

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
#create new input
new_input=np.arange(1, 100, 1).reshape(-1, 1)

#fit the model
lm.fit(X, Y)
lm

#produce a prediction
yhat=lm.predict(new_input)
yhat[0:5]

#plot a data
plt.plot(new_input, yhat)
plt.show()

<img src="prediction_to_predict.png">

#### Decision Making
Even tho multiple models usually results better, we need to check the MSE and R^2 to convinced